In [1]:
!git clone https://github.com/huggingface/diffusers.git
!pip install ./diffusers
!pip install -U -r ./diffusers/examples/dreambooth/requirements.txt

Cloning into 'diffusers'...
remote: Enumerating objects: 47658, done.
remote: Counting objects: 100% (1505/1505), done.
remote: Compressing objects: 100% (699/699), done.
remote: Total 47658 (delta 977), reused 1094 (delta 681), pack-reused 46153
Receiving objects: 100% (47658/47658), 31.87 MiB | 2.03 MiB/s, done.
Resolving deltas: 100% (35119/35119), done.
Processing ./diffusers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.25.0.dev0-py3-none-any.whl size=1816331 sha256=b70b693446b73bf19dd06c22b8056d4359683599abdb1a394bc0a6a5a3c8929d
  Stored in directory: /tmp/pip-ephem-wheel-cache-_0wd2u5e/wheels/64/78/12/760ac642f0ef9ca736fa88da48b7461adddf1b24155a244ddc
Successfully built diffusers
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.25.0.dev0
    Uninstalling diffusers-0.25.0.dev0:
      Successfully uninstalled dif

In [1]:
!accelerate config default --mixed_precision fp16

Configuration already exists at /root/.cache/huggingface/accelerate/default_config.yaml, will not override. Run `accelerate config` manually or pass a different `save_location`.


In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv


name, memory.total [MiB], memory.free [MiB]
NVIDIA GeForce RTX 4090, 24564 MiB, 11295 MiB


In [4]:
import os

os.environ['MODEL_NAME'] = f'CompVis/stable-diffusion-v1-4'
os.environ['INSTANCE_DIR'] = './bash_images'
os.environ['OUTPUT_DIR'] = f'./dreambooth/sd_aiconos-model-v1-2_400'

In [5]:
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv


name, memory.total [MiB], memory.free [MiB]
NVIDIA GeForce RTX 4090, 24564 MiB, 11295 MiB


In [7]:
import torch
torch.cuda.empty_cache()
!accelerate launch ./diffusers/examples/dreambooth/train_dreambooth.py \
  --pretrained_model_name_or_path=$MODEL_NAME  \
  --instance_data_dir=$INSTANCE_DIR \
  --seed=1337 \
  --output_dir=$OUTPUT_DIR \
  --revision="fp16" \
  --mixed_precision="fp16" \
  --instance_prompt="a photo of bash" \
  --resolution=8 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --checkpointing_steps=100 \
  --learning_rate=1e-4 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=10

12/12/2023 12:16:06 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: fp16

You are using a model of type clip_text_model to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
{'timestep_spacing', 'clip_sample_range', 'variance_type', 'prediction_type', 'sample_max_value', 'thresholding', 'dynamic_thresholding_ratio'} was not found in config. Values will be initialized to default values.
/usr/local/lib/python3.10/dist-packages/diffusers/utils/hub_utils.py:302: FutureWarning: You are loading the variant fp16 from CompVis/stable-diffusion-v1-4 via `revision='fp16'`. This behavior is deprecated and will be removed in diffusers v1. One should use `variant='fp16'` instead. However, it appears that CompVis/stable-diffusion-v1-4 currently does not have a diffusion_pytorch_model.fp16.safetensors file in the 'main' branch of CompVis/stable-diffusion-